* User feature như sau:
+ User embed
+ User mua bao nhiêu sản phẩm
+ User tỷ lệ mua bao nhiêu
* Item feature như sau:
+ Item embed
+ Item được tương tác bao lần
+ Item hay được mua cùng như nào (Co-visitation matrix)
+ Item có tỷ lệ mua là bao nhiêu
* User-Item feature
+ score embedding từ 4 model

Tạo candidate bằng cách lấy dữ liệu thật có trường rating, negative sample bằng XSimGCL 
Tập test là tập user id trong test
Infer bằng cách lấy top 20 sau khi rerank bằng XSimGCL rerank lại hoặc lấy luôn 1000 sản phẩm rerank

In [4]:
import torch
import warnings
import gc
import logging
from tqdm import tqdm
import os
import glob
import csv
import pandas as pd
warnings.filterwarnings('ignore')

In [5]:
predict_df = pd.read_csv('data/test_set_private.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
test_user_id = predict_df['user_id'].values

In [ ]:
# !pip install gitpython --quiet
# import git
# git.Repo.clone_from('https://github.com/Coder-Yu/SELFRec.git', '/kaggle/working/sample')

In [5]:
# cd /kaggle/working/sample

In [6]:
user_embedding_simgcl = pd.read_pickle('/kaggle/input/weight-model/user_embedding_simgcl.pkl')
user_embedding_lightgcn = pd.read_pickle('/kaggle/input/weight-model/user_embedding_lightgcn.pkl')
# user_embedding_recvae = pd.read_pickle('/kaggle/input/weight-model/user_embedding_recvae.pkl')
user_embedding_xsim = pd.read_pickle('/kaggle/input/weight-model/emb_user_XSim_full.pickle')
user_embedding_xsim_standard = pd.read_pickle('/kaggle/input/weight-model/emb_user_XSim_standard.pickle')
user_embedding_xsim_denoise = pd.read_pickle('/kaggle/input/weight-model/emb_user_XSim_denoise.pickle')
item_embedding_simgcl = pd.read_pickle('/kaggle/input/weight-model/item_embedding_simgcl.pkl')
item_embedding_lightgcn = pd.read_pickle('/kaggle/input/weight-model/item_embedding_lightgcn.pkl')
# item_embedding_recvae = pd.read_pickle('/kaggle/input/weight-model/item_embedding_recvae.pkl')
item_embedding_xsim = pd.read_pickle('/kaggle/input/weight-model/emb_item_XSim_full.pickle')
item_embedding_xsim_standard = pd.read_pickle('/kaggle/input/weight-model/emb_item_XSim_standard.pickle')
item_embedding_xsim_denoise = pd.read_pickle('/kaggle/input/weight-model/emb_item_XSim_denoise.pickle')

In [7]:
buy_order_full = pd.read_csv('/kaggle/input/covisitation-matrix/buy_order_full.csv')
buy_order_standard = pd.read_csv('/kaggle/input/covisitation-matrix/buy_order_standard.csv')
cart_order_full = pd.read_csv('/kaggle/input/covisitation-matrix/cart_order_full.csv')
cart_order_standard = pd.read_csv('/kaggle/input/covisitation-matrix/cart_order_standard.csv')

In [8]:
buy_order_full = buy_order_full.groupby('ItemId_x')['wgt'].sum().reset_index().rename(columns={'ItemId_x': 'ItemId', 'wgt': 'buy_order_full'})
buy_order_standard = buy_order_standard.groupby('ItemId_x')['wgt'].sum().reset_index().rename(columns={'ItemId_x': 'ItemId', 'wgt': 'buy_order_standard'})
cart_order_full = cart_order_full.groupby('ItemId_x')['wgt'].sum().reset_index().rename(columns={'ItemId_x': 'ItemId','wgt': 'cart_order_full'})
cart_order_standard = cart_order_standard.groupby('ItemId_x')['wgt'].sum().reset_index().rename(columns={'ItemId_x': 'ItemId','wgt': 'cart_order_standard'})

In [9]:
user_embedding_xsim_128 = pd.read_pickle('/kaggle/input/weight-model/emb_user_XSim_128.pickle')
item_embedding_xsim_128 = pd.read_pickle('/kaggle/input/weight-model/emb_item_XSim128.pickle')

In [10]:
def convert_embeddings_to_df(id_name, embeddings_dict):
    """
    Chuyển đổi dictionary chứa id và vector embedding thành pandas dataframe
    
    Parameters:
    embeddings_dict (dict): Dictionary với key là id và value là vector embedding 128 chiều
    
    Returns:
    pandas.DataFrame: DataFrame với 129 cột (1 cột id và 128 cột cho vector embedding)
    """
    # Tạo list các records, mỗi record là list chứa id và 128 giá trị embedding
    records = []
    for id_, embedding in embeddings_dict.items():
        record = [id_] + embedding.tolist() if isinstance(embedding, np.ndarray) else [id_] + embedding
        records.append(record)
    
    # Tạo tên cho các cột
    columns = [id_name] + [f'dim_{id_name}_{i}' for i in range(128)]
    
    # Tạo DataFrame
    df = pd.DataFrame(records, columns=columns)
    
    return df

In [11]:
item_embed_feature = convert_embeddings_to_df('ItemId',item_embedding_xsim_128)
user_embed_feature = convert_embeddings_to_df('UserId',user_embedding_xsim_128)

In [12]:
user_emb = []
user2id = {}
cnt = 0
user_list_xsim = []
for user in user_embedding_xsim:
    user_emb.append(user_embedding_xsim[user])
    user_list_xsim.append(user)
    user2id[user] = cnt
    cnt+=1
user_emb = torch.tensor(user_emb)

item_emb = []
item2id = {}
cnt = 0
item_list_xsim = []
for item in item_embedding_xsim:
    item_emb.append(item_embedding_xsim[item])
    item_list_xsim.append(item)
    item2id[item] = cnt
    cnt+=1
item_emb = torch.tensor(item_emb)

In [13]:
len(user2id), len(item2id)

(36751, 82673)

In [14]:
data = pd.read_csv('data/training_set.csv')
data.shape, data.UserId.nunique(), data.ItemId.nunique()

((389923, 4), 36751, 83102)

In [15]:
user_rated = data.groupby('UserId')['ItemId'].apply(list).to_dict()

In [16]:
def find_k_largest(K, candidates):
    n_candidates = []
    for iid, score in enumerate(candidates[:K]):
        n_candidates.append((score, iid))
    heapq.heapify(n_candidates)
    for iid, score in enumerate(candidates[K:]):
        if score > n_candidates[0][0]:
            heapq.heapreplace(n_candidates, (score, iid + K))
    n_candidates.sort(key=lambda d: d[0], reverse=True)
    ids = [item[1] for item in n_candidates]
    k_largest_scores = [item[0] for item in n_candidates]
    return ids, k_largest_scores
    
def generate_candidate(user_id_list):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        print(f'\rProgress: [{"+" * ratenum}{" " * (50 - ratenum)}]{ratenum * 2}%', end='', flush=True)

    rec_list = {}
    user_count = len(user_id_list)
    for i, user in enumerate(user_id_list):
        candidates = torch.matmul(torch.tensor(user_embedding_xsim[user]), torch.tensor(item_emb.transpose(0, 1))).cpu().numpy()
        rated_list = user_rated[user]
        for item in rated_list:
            try:
                candidates[item2id[item]] = -10e8
            except:
                pass
        ids, scores = find_k_largest(100, candidates)
        item_names = [item_list_xsim[iid] for iid in ids]
        rec_list[user] = list(zip(item_names, scores))
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    return rec_list

In [17]:
# rec_list = generate_candidate(user_list_xsim[:10000])

In [18]:
import pickle

# with open('rec_list.pickle', 'wb') as handle:
#     pickle.dump(rec_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/kaggle/input/weight-model/rec_list.pickle', 'rb') as handle:
    rec_list = pickle.load(handle)

In [19]:
data_sub_stage1 = pd.read_csv('/kaggle/input/submission/predict_ensemble_7file.csv', names = ['UserId'] + [f'item_id_{i}' for i in range(1,1001)])
data_sub_stage1 = data_sub_stage1[['UserId']+[f'item_id_{i}' for i in range(1,16)]]

In [20]:
# Chuyển từ dạng wide sang long
df_long = data_sub_stage1.melt(id_vars=['UserId'], value_vars=[col for col in data_sub_stage1.columns if col.startswith('item_id')], 
                  var_name='item_col', value_name='ItemId')
df_submit = df_long.drop(columns=['item_col'])
df_submit

,UserId,ItemId
0,0a9Fr9I8Gr,EALxnioXfL
1,Jg3UwpR1cA,Pyze1t4kzV
2,k0mNOjn7pv,jAE0cnq4rG
3,Uw0zXc8Sr8,S41d9ELmhC
4,vLDKNL74C4,PMsUraEglo
...,...,...
63370,SIFzQe6gnn,0krwZa3Mav
63371,fa6Y1KVdoh,AgS4FFBg8m
63372,xO9WRD2Zbm,FoRJXdWPFm
63373,EW403CecBe,TEBViQF2Di


In [21]:
item_features = data.groupby('ItemId').agg({'UserId':'nunique','Purchase':'mean'})
item_features.columns = ['item_user_count','item_buy_ratio']

user_features = data.groupby('UserId').agg({'ItemId':'nunique','Purchase':'mean'})
user_features.columns = ['user_item_count','user_buy_ratio']

In [22]:
def convert_dict_to_dataframe(input_dict):
    """
    Convert a dictionary with nested tuples to a DataFrame
    
    Args:
        input_dict (dict): Dictionary with user_id as key and list of (item_id, score) tuples as value
    
    Returns:
        pandas.DataFrame: DataFrame with user_id and item_id columns
    """
    # Prepare lists to store data
    user_ids = []
    item_ids = []
    
    # Iterate through the dictionary
    for user_id, items in input_dict.items():
        for item_id, _ in items[:15]:
            user_ids.append(user_id)
            item_ids.append(item_id)
    
    # Create DataFrame
    df = pd.DataFrame({
        'UserId': user_ids,
        'ItemId': item_ids
    })
    
    return df

negative_sample = convert_dict_to_dataframe(rec_list)
negative_sample['Click'] = 0
negative_sample['Purchase'] = 0

In [23]:
data_rerank = pd.concat([data, negative_sample])
# # Chuyển embeddings thành array để dễ tính toán
# embedding_dim = len(next(iter(item_emb.values())))  # = 1024
    
# Tìm item cuối cùng có rating = 1 cho mỗi user
last_items = (data_rerank[data_rerank['Click'] >0]
                 .sort_values('UserId')
                 .groupby('UserId')
                 .last()
                 .reset_index()
                 [['UserId', 'ItemId']])
    
# Tạo dictionary để map user_id với last_item_id
user_last_item = dict(zip(last_items['UserId'], last_items['ItemId']))

In [24]:
data_rerank.to_csv('input.csv', index = None)

In [25]:
class ScoreCalculator:
    def __init__(self, embedding_configs, user_last_item, batch_size=1000):
        """
        Parameters:
        -----------
        embedding_configs: dict
            Cấu hình embeddings cho các model
        user_last_item: dict
            Dictionary mapping user đến item cuối cùng
        batch_size: int
            Số lượng rows xử lý mỗi batch
        """
        self.embedding_configs = embedding_configs
        self.user_last_item = user_last_item
        self.batch_size = batch_size
        # Initialize feature_columns with all possible columns at init
        self.feature_columns = []
        # Pre-generate all score column names
        for model_name in self.embedding_configs:
            self.feature_columns.extend([
                f'score_{model_name}_dot',
                f'score_{model_name}_cosine',
                f'score_item_{model_name}_dot',
                f'score_item_{model_name}_cosine'
            ])

    def _create_score_columns(self, df):
        """Khởi tạo các cột score"""
        # Initialize all columns with 0.0
        for col in self.feature_columns:
            df[col] = 0.0
        return df

    def _process_batch(self, batch_df, user_matrix, item_matrix, user_to_idx, item_to_idx, 
                      user_last_items, model_name):
        """Xử lý một batch của DataFrame"""
        try:
            user_indices = batch_df['UserId'].map(user_to_idx).values
            item_indices = batch_df['ItemId'].map(item_to_idx).values

            # Lấy embeddings cho batch hiện tại
            batch_user_embeddings = user_matrix[user_indices]
            batch_item_embeddings = item_matrix[item_indices]

            # Tính dot product scores
            dot_scores = np.sum(batch_user_embeddings * batch_item_embeddings, axis=1)
            batch_df[f'score_{model_name}_dot'] = dot_scores

            # Tính cosine similarity với last item
            valid_users = batch_df['UserId'].isin(self.user_last_item)
            if valid_users.any():
                last_item_embeddings = item_matrix[user_last_items[user_indices]]
                
                # Normalize và tính cosine similarity
                last_item_norm = np.linalg.norm(last_item_embeddings, axis=1, keepdims=True)
                current_item_norm = np.linalg.norm(batch_item_embeddings, axis=1, keepdims=True)
                
                cosine_scores = np.sum(
                    (last_item_embeddings / np.maximum(last_item_norm, 1e-8)) * 
                    (batch_item_embeddings / np.maximum(current_item_norm, 1e-8)),
                    axis=1
                )

                batch_df.loc[valid_users, f'score_{model_name}_cosine'] = cosine_scores[valid_users]
                batch_df.loc[valid_users, f'score_item_{model_name}_dot'] = np.sum(
                    last_item_embeddings[valid_users] * batch_item_embeddings[valid_users], 
                    axis=1
                )
                batch_df.loc[valid_users, f'score_item_{model_name}_cosine'] = cosine_scores[valid_users]

            # Giải phóng bộ nhớ
            del batch_user_embeddings, batch_item_embeddings
            gc.collect()

        except Exception as e:
            logger.error(f"Error processing batch for {model_name}: {str(e)}")
            
        return batch_df

    def calculate_scores(self, df):
        """
        Tính toán scores theo batch để tiết kiệm RAM
        """
        logger.info("Starting score calculation...")
        result_df = df.copy()
        
        # Tính rating
        result_df['rating'] = result_df['Click'] 
        result_df = result_df.drop(['Click', 'Purchase'], axis=1)
        
        # Khởi tạo cột scores
        result_df = self._create_score_columns(result_df)
        
        # Xử lý từng model
        for model_name, embeddings in self.embedding_configs.items():
            logger.info(f"Processing model: {model_name}")
            
            try:
                # Chuẩn bị embeddings
                unique_users = result_df['UserId'].unique()
                unique_items = result_df['ItemId'].unique()
                
                user_to_idx = {uid: idx for idx, uid in enumerate(unique_users)}
                item_to_idx = {iid: idx for idx, iid in enumerate(unique_items)}
                
                # Tạo user last items array
                user_last_items = np.zeros(len(unique_users), dtype=int)
                for uid, last_iid in self.user_last_item.items():
                    if uid in user_to_idx and last_iid in item_to_idx:
                        user_last_items[user_to_idx[uid]] = item_to_idx[last_iid]
                
                # Load embeddings vào matrix
                dim = len(next(iter(embeddings['user_embedding'].values())))
                user_matrix = np.zeros((len(unique_users), dim))
                item_matrix = np.zeros((len(unique_items), dim))
                
                for uid, emb in embeddings['user_embedding'].items():
                    if uid in user_to_idx:
                        user_matrix[user_to_idx[uid]] = emb
                        
                for iid, emb in embeddings['item_embedding'].items():
                    if iid in item_to_idx:
                        item_matrix[item_to_idx[iid]] = emb
                
                # Xử lý theo batch
                num_batches = len(result_df) // self.batch_size + 1
                for i in tqdm(range(num_batches), desc=f"Processing {model_name}"):
                    start_idx = i * self.batch_size
                    end_idx = min((i + 1) * self.batch_size, len(result_df))
                    batch_df = result_df.iloc[start_idx:end_idx].copy()
                    
                    batch_df = self._process_batch(
                        batch_df, user_matrix, item_matrix,
                        user_to_idx, item_to_idx, user_last_items,
                        model_name
                    )
                    
                    # Cập nhật kết quả
                    result_df.iloc[start_idx:end_idx] = batch_df
                    
                    # Giải phóng bộ nhớ
                    del batch_df
                    gc.collect()
                
                # Giải phóng bộ nhớ sau khi xử lý xong một model
                del user_matrix, item_matrix
                gc.collect()
                
            except Exception as e:
                logger.error(f"Error processing model {model_name}: {str(e)}")
                continue
        
        logger.info("Score calculation completed")
        # Ensure consistent column order
        return result_df[['UserId', 'ItemId', 'rating'] + self.feature_columns]


def process_dataframe_in_chunks(input_path, output_dir, final_output_path, embedding_configs, 
                              user_last_item, chunk_size=10000, batch_size=1000):
    """
    Xử lý DataFrame theo chunks và lưu thành các file CSV riêng
    """
    calculator = ScoreCalculator(embedding_configs, user_last_item, batch_size)
    
    # Tạo thư mục output nếu chưa tồn tại
    os.makedirs(output_dir, exist_ok=True)
    
    # Xóa các file CSV cũ trong thư mục output (nếu có)
    for f in glob.glob(os.path.join(output_dir, "chunk_*.csv")):
        os.remove(f)
    
    try:
        # Đọc và xử lý từng chunk
        chunk_files = []
        for chunk_idx, chunk in enumerate(pd.read_csv(input_path, chunksize=chunk_size)):
            logger.info(f"Processing chunk {chunk_idx + 1}")
            
            # Tính scores cho chunk
            result_chunk = calculator.calculate_scores(chunk)
            
            # Verify column count
            expected_cols = 3 + len(calculator.feature_columns)  # UserId, ItemId, rating + feature columns
            if len(result_chunk.columns) != expected_cols:
                raise ValueError(f"Unexpected number of columns in chunk {chunk_idx}: "
                               f"got {len(result_chunk.columns)}, expected {expected_cols}")
            
            # Lưu chunk vào file CSV
            chunk_path = os.path.join(output_dir, f"chunk_{chunk_idx:04d}.csv")
            result_chunk.to_csv(chunk_path, index=False)
            chunk_files.append(chunk_path)
            
            # In thông tin debug
            logger.info(f"Chunk {chunk_idx} columns: {list(result_chunk.columns)}")
            
            # Giải phóng bộ nhớ
            del result_chunk
            gc.collect()
        
        logger.info("Merging CSV files...")
        
        # Đọc header từ chunk đầu tiên
        first_chunk = pd.read_csv(chunk_files[0], nrows=0)
        columns = first_chunk.columns.tolist()
        
        # Ghi file cuối cùng
        with open(final_output_path, 'w', newline='') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(columns)
            
            for chunk_file in tqdm(chunk_files, desc="Merging files"):
                with open(chunk_file, 'r', newline='') as infile:
                    next(infile)  # Skip header
                    for line in infile:
                        outfile.write(line)
                        
                try:
                    os.remove(chunk_file)
                except Exception as e:
                    logger.warning(f"Could not remove temporary file {chunk_file}: {str(e)}")
        
        logger.info(f"Final results saved to: {final_output_path}")
        return final_output_path
        
    except Exception as e:
        logger.error(f"Error during processing: {str(e)}")
        # Cleanup temp files
        for f in glob.glob(os.path.join(output_dir, "chunk_*.csv")):
            try:
                os.remove(f)
            except:
                pass
        raise e

In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gc
import logging
from tqdm import tqdm

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Cách sử dụng:
embedding_configs = {
    'simgcl': {
        'user_embedding': user_embedding_simgcl,
        'item_embedding': item_embedding_simgcl
    },
    'lightgcn': {
        'user_embedding': user_embedding_lightgcn,
        'item_embedding': item_embedding_lightgcn
    },
    'xsim': {
        'user_embedding': user_embedding_xsim,
        'item_embedding': item_embedding_xsim
    },
    'xsim_denoise': {
        'user_embedding': user_embedding_xsim_denoise,
        'item_embedding': item_embedding_xsim_standard
    },
    'xsim_standard': {
        'user_embedding': user_embedding_xsim_standard,
        'item_embedding': item_embedding_xsim_standard
    }
}
# Xử lý dữ liệu
final_csv_path = process_dataframe_in_chunks(
    input_path='input.csv',
    output_dir='temp_csv',     # Thư mục chứa các file CSV tạm thời
    final_output_path='output.csv',  # File CSV cuối cùng
    embedding_configs=embedding_configs,
    user_last_item=user_last_item,
    chunk_size=400000,  # Số rows mỗi chunk
    batch_size=400000   # Số rows mỗi batch trong tính toán
)

# # Nếu muốn đọc kết quả:
# # result_df = pd.read_csv(final_csv_path)


Merging files: 100%|██████████| 3/3 [00:01<00:00,  2.53it/s]


In [27]:
item_embed_feature['ItemId'] = item_embed_feature['ItemId'].astype(str)
user_embed_feature['UserId'] = user_embed_feature['UserId'].astype(str)

In [28]:
result_df = pd.read_csv(final_csv_path)
result_df = result_df.merge(item_features, left_on='ItemId', right_index=True, how='left').fillna(0)
result_df = result_df.merge(user_features, left_on='UserId', right_index=True, how='left').fillna(0)
result_df = result_df.merge(item_embed_feature, on='ItemId', how='left').fillna(0)
result_df = result_df.merge(user_embed_feature, on='UserId', how='left').fillna(0)
result_df = result_df.merge(buy_order_full, on='ItemId', how='left').fillna(0)
result_df = result_df.merge(buy_order_standard, on='ItemId', how='left').fillna(0)
result_df = result_df.merge(cart_order_full, on='ItemId', how='left').fillna(0)
result_df = result_df.merge(cart_order_standard, on='ItemId', how='left').fillna(0)

In [29]:
# for i in ['item_user_count', 'user_item_count']:
#     data_rerank[i] = (data_rerank[i] - data_rerank[i].min())/(data_rerank[i].max()-data_rerank[i].min())

In [30]:
df_submit['Click'] = 0
df_submit['Purchase'] = 0
df_submit.to_csv('input_submit.csv', index = None)

submit_csv_path = process_dataframe_in_chunks(
    input_path='input_submit.csv',
    output_dir='temp_csv',     # Thư mục chứa các file CSV tạm thời
    final_output_path='output_submit.csv',  # File CSV cuối cùng
    embedding_configs=embedding_configs,
    user_last_item=user_last_item,
    chunk_size=400000,  # Số rows mỗi chunk
    batch_size=400000   # Số rows mỗi batch trong tính toán
)

Merging files: 100%|██████████| 1/1 [00:00<00:00, 12.39it/s]


In [31]:
df_submit = pd.read_csv(submit_csv_path)
df_submit = df_submit.merge(item_features, left_on='ItemId', right_index=True, how='left').fillna(0)
df_submit = df_submit.merge(user_features, left_on='UserId', right_index=True, how='left').fillna(0)
df_submit = df_submit.merge(item_embed_feature, on='ItemId', how='left').fillna(0)
df_submit = df_submit.merge(user_embed_feature, on='UserId', how='left').fillna(0)
df_submit = df_submit.merge(buy_order_full, on='ItemId', how='left').fillna(0)
df_submit = df_submit.merge(buy_order_standard, on='ItemId', how='left').fillna(0)
df_submit = df_submit.merge(cart_order_full, on='ItemId', how='left').fillna(0)
df_submit = df_submit.merge(cart_order_standard, on='ItemId', how='left').fillna(0)

In [32]:
result_df_nostandard  = result_df.copy()

In [33]:
for i in ['score_simgcl_dot', 'score_simgcl_cosine',
       'score_item_simgcl_dot', 'score_item_simgcl_cosine',
       'score_lightgcn_dot', 'score_lightgcn_cosine',
       'score_item_lightgcn_dot', 'score_item_lightgcn_cosine',
       'score_xsim_dot', 'score_xsim_cosine', 'score_item_xsim_dot',
       'score_item_xsim_cosine', 'score_xsim_denoise_dot',
       'score_xsim_denoise_cosine', 'score_item_xsim_denoise_dot',
       'score_item_xsim_denoise_cosine', 'score_xsim_standard_dot',
       'score_xsim_standard_cosine', 'score_item_xsim_standard_dot',
       'score_item_xsim_standard_cosine']:
    result_df[i] = (result_df[i] - result_df[i].min())/(result_df[i].max()-result_df[i].min())

In [34]:
for i in ['score_simgcl_dot', 'score_simgcl_cosine',
       'score_item_simgcl_dot', 'score_item_simgcl_cosine',
       'score_lightgcn_dot', 'score_lightgcn_cosine',
       'score_item_lightgcn_dot', 'score_item_lightgcn_cosine',
       'score_xsim_dot', 'score_xsim_cosine', 'score_item_xsim_dot',
       'score_item_xsim_cosine', 'score_xsim_denoise_dot',
       'score_xsim_denoise_cosine', 'score_item_xsim_denoise_dot',
       'score_item_xsim_denoise_cosine', 'score_xsim_standard_dot',
       'score_xsim_standard_cosine', 'score_item_xsim_standard_dot',
       'score_item_xsim_standard_cosine']:
    df_submit[i] = (df_submit[i] - df_submit[i].min())/(df_submit[i].max()-df_submit[i].min())

In [35]:
test = result_df[result_df.UserId.isin(test_user_id)]
train = result_df.drop(test.index)
# infer = df_test_submit.copy()

In [36]:
train_baskets = train.groupby(['UserId'])['ItemId'].count().values

In [37]:
%%time
# columns_to_use = ['score_simgcl_dot', 'score_simgcl_cosine',
#        'score_item_simgcl_dot', 'score_item_simgcl_cosine',
#        'score_lightgcn_dot', 'score_lightgcn_cosine',
#        'score_item_lightgcn_dot', 'score_item_lightgcn_cosine',
#        'score_xsim_dot', 'score_xsim_cosine', 'score_item_xsim_dot',
#        'score_item_xsim_cosine', 'score_xsim_denoise_dot',
#        'score_xsim_denoise_cosine', 'score_item_xsim_denoise_dot',
#        'score_item_xsim_denoise_cosine', 'score_xsim_standard_dot',
#        'score_xsim_standard_cosine', 'score_item_xsim_standard_dot',
#        'score_item_xsim_standard_cosine', 'item_user_count', 'item_buy_ratio',
#        'user_item_count', 'user_buy_ratio']
columns_to_use = train.columns[3:]
train_X = train[columns_to_use]
train_y = train['rating']

test_X = test[columns_to_use]

CPU times: user 333 ms, sys: 396 ms, total: 728 ms
Wall time: 727 ms


In [38]:
from lightgbm.sklearn import LGBMRanker
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=150,
    importance_type='gain',
    verbose=10,
    learning_rate = 0.05,
    # max_depth = 8,
    # num_leaves = 10,
)

In [39]:
%%time
ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.827417
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.010277
[LightGBM] [Debug] init for col-wise cost 0.016390 seconds, init for row-wise cost 1.245520 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.779946 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 71816
[LightGBM] [Info] Number of data points in the train set: 790906, number of used features: 284
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 6
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 6
[LightGBM] [Warning] No further 

In [40]:
# import lightgbm as lgb
# clf = lgb.LGBMClassifier(class_weight = {0: 0.3, 1:0.7})
# clf.fit(train_X, train_y)

In [41]:
cnt = 0
feature_important = []
for i in ranker.feature_importances_.argsort()[::-1]:
    cnt+=1
    feature_important.append(columns_to_use[i])
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())
    if cnt == 50:
        break

score_simgcl_cosine 0.9356504884533833
user_item_count 0.06434951154661094
score_simgcl_dot 2.3655661275034325e-15
dim_ItemId_20 9.050629284897327e-16
item_buy_ratio 3.00312108421983e-16
dim_ItemId_11 2.1886379623250216e-16
score_xsim_denoise_cosine 2.1358094998053093e-16
dim_ItemId_91 2.0355088789904715e-16
score_xsim_dot 1.609720929318483e-16
dim_ItemId_21 9.872232631535667e-17
dim_ItemId_37 9.726042995834073e-17
score_lightgcn_dot 9.702450537878968e-17
dim_ItemId_9 8.353930940505659e-17
dim_ItemId_50 5.777153448658463e-17
dim_ItemId_10 4.16909395629392e-17
dim_ItemId_35 3.885978385709606e-17
dim_ItemId_8 3.685202993110934e-17
dim_ItemId_102 3.636165200181237e-17
score_item_xsim_dot 3.617200921476645e-17
dim_ItemId_2 3.2582116397267357e-17
dim_ItemId_105 3.1865022391870075e-17
dim_ItemId_123 3.130984342782347e-17
score_xsim_cosine 2.9366932213165936e-17
dim_ItemId_60 2.8016098217365157e-17
dim_ItemId_4 2.7405424148993242e-17
dim_ItemId_112 2.7183386463620146e-17
dim_ItemId_40 2.47037

In [42]:
import math

def calculate_ndcg(predictions: dict, labels: dict, N: int = None) -> float:
    """
    Tính NDCG cho predicted rankings so với ground truth labels
    
    Args:
        predictions (dict): Dictionary chứa predicted rankings của mỗi user
        labels (dict): Dictionary chứa ground truth items của mỗi user
        N (int): Số lượng items tối đa để xét (NDCG@N)
    
    Returns:
        float: Giá trị NDCG trung bình của tất cả users
    """
    sum_ndcg = 0
    
    for user_id in predictions:
        if user_id not in labels:
            continue
            
        # Tính DCG cho predicted ranking
        dcg = 0
        for pos, item in enumerate(predictions[user_id][:N]):
            # Kiểm tra xem item có trong labels không
            if item in labels[user_id]:
                dcg += 1.0 / math.log2(pos + 2)
        
        # Tính IDCG cho ground truth
        idcg = 0
        for pos in range(min(N, len(labels[user_id]))):
            idcg += 1.0 / math.log2(pos + 2)
        
        # Cộng dồn NDCG của user hiện tại
        if idcg != 0:
            sum_ndcg += dcg / idcg
    
    # Tính trung bình NDCG của tất cả users
    avg_ndcg = round(sum_ndcg / len(predictions), 5)
    
    return avg_ndcg

In [43]:
test['pred'] = ranker.predict(test_X)
tmp_test = test \
    .sort_values(['UserId', 'pred'], ascending=False) \
    .groupby('UserId')['ItemId'].apply(list).to_dict()
label = test[test.rating != 0].sort_values(['UserId', 'rating'], ascending=False).groupby('UserId')['ItemId'].apply(list).to_dict()


In [44]:
calculate_ndcg(tmp_test, label, 5)

0.68835

In [45]:
columns_to_use = feature_important
train_X = train[columns_to_use]
train_y = train['rating']

test_X = test[columns_to_use]

In [46]:
%%time
from lightgbm.sklearn import LGBMRanker
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=150,
    importance_type='gain',
    verbose=10,
    learning_rate = 0.05,
    # max_depth = 8,
    # num_leaves = 10,
)
ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets
)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.011876
[LightGBM] [Debug] init for col-wise cost 0.000010 seconds, init for row-wise cost 0.182148 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.244226 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12563
[LightGBM] [Info] Number of data points in the train set: 790906, number of used features: 50
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 6
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 6
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with

In [47]:
# for i in clf.feature_importances_.argsort()[::-1]:
#     print(columns_to_use[i], clf.feature_importances_[i]/clf.feature_importances_.sum(), clf.feature_importances_[i])

In [48]:
test['pred'] = ranker.predict(test_X)
tmp_test = test \
    .sort_values(['UserId', 'pred'], ascending=False) \
    .groupby('UserId')['ItemId'].apply(list).to_dict()
label = test[test.rating != 0].sort_values(['UserId', 'rating'], ascending=False).groupby('UserId')['ItemId'].apply(list).to_dict()


In [49]:
calculate_ndcg(tmp_test, label, 5)

0.5877

In [50]:
df_submit['pred'] = ranker.predict(df_submit[columns_to_use])
# test['pred'] = clf.predict_proba(test_X)[:,1]

In [51]:
tmp_submit = df_submit \
    .sort_values(['UserId', 'pred'], ascending=False) \
    .groupby('UserId')['ItemId'].apply(list).to_dict()

In [52]:
submit = pd.read_csv('/kaggle/input/vhac-recsys/test_set_private.csv', names = ['user_id']+[f'item_id_{i}' for i in range(1, 1001)])

In [53]:
import warnings
warnings.filterwarnings('ignore')

In [54]:
for i in range(len(submit)):
    uid = submit.loc[i, 'user_id']
    list_item =  tmp_submit[uid] 
    for j in range(len(list_item)):
        submit.loc[i, f'item_id_{j+1}'] = list_item[j]

In [55]:
submit.to_csv('predict.csv', header = None, index = None)

In [56]:
submit

,user_id,item_id_1,item_id_2,item_id_3,item_id_4,item_id_5,item_id_6,item_id_7,item_id_8,item_id_9,...,item_id_991,item_id_992,item_id_993,item_id_994,item_id_995,item_id_996,item_id_997,item_id_998,item_id_999,item_id_1000
0,0a9Fr9I8Gr,Rw50zQ6IiC,sLoTA2aPEl,5I3jXuSXJC,ZXR0UOYMC5,4nfx37EgS2,h6s6cG6xcZ,TiKwggguXm,6pPEL58mZO,PRanViElTN,...,Z8xiRnRjZD,5MwdFYt8z0,IzLVUKs7uQ,Ic5BpxKXZg,F9jblvzWuK,xvKZji6pWJ,arvytMaCpS,XC7SvTAElR,oWsTKxTtvA,iLKSmk2T06
1,Jg3UwpR1cA,Y9CejSQQoa,IFADrTRXqz,9P4pCVzEhe,nzXfQ59ZOp,dYzjsJalSw,btJqToDrNx,5g0f4SiT73,imJxX0LSMe,Pyze1t4kzV,...,84YaBhvrnw,uUg3iZKEDz,VS9vm70j8u,mb8iada716,W4LaO2AnPF,oH6lYVrH2Z,RdMLs1XP8z,cjhX33aANo,JVR40qZevF,m5tbhqzC7m
2,k0mNOjn7pv,tej89WKt6t,0iC5hrr8pF,KCSRaARC9s,1i0eG8FdZX,osGyK5E0G3,wbKXQYFoKH,i1D3Eq2Pb7,cGAltvZgDp,UokCuNyD2b,...,LcT9aU6sDk,erjkmeJrO2,DtAWzeNoqo,9C1LcSgNrd,RfaOiwln47,EjaSqbgfWm,5MmscAd5ND,4CDZjAxiJ3,iPzOhrroRk,Or6J4bnaJo
3,Uw0zXc8Sr8,9H1pnwbhUi,iiw3cObpDv,HydOLulMTb,U3fKcoE1Ft,S41d9ELmhC,4hALWkoqeJ,dyVIXkivM9,7f7LkOKsAr,Daytgu5kmV,...,UmYJDzVAm1,6c9wVM3C7n,yy78DDyFwz,JBWS6uOW6t,uFtLv6hcKO,Z8p8xxx0cx,Py42KZCHJP,ewfqfR6fiC,rdZDIeJ1ia,OgmtWQ39sq
4,vLDKNL74C4,1E0dDrSxY9,CkcNFCUlTC,UAwDMDYT41,iEWXEiSBg5,U3cH0WiMF6,Mu7TTk6ZXo,PPx7lWosOF,gQCGJIJOaM,uzts0Vtcry,...,MUWZaAPDNT,rnp6yH6OD5,45KRQI5RJi,iPRCkgoeGJ,6PtfgezcfB,w83nfFLEQF,aD2c2V2wYx,Cu2tQGKjrm,Ec3YhNEIRi,ax59cfAF2Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4220,SIFzQe6gnn,32FfxVfh3i,1Qkjg4RArI,l6DuJC6Rg2,0krwZa3Mav,BndaktM5jy,4Pm47rVPdy,2BQ2BW84WZ,8f9Vtw6DqP,Bc5My2qvfP,...,h43ef0UCse,9BC1n9G6rS,0dVp4MRu5T,GoxXzgB7Q0,kavQ1mOVNr,1hIY7i8r8j,eqHobu9Yqj,TvY4Mry9Jq,FMbHiyRS9j,Z8DZIlk6Ba
4221,fa6Y1KVdoh,LTEBOO65ld,S6L5JSHYt2,iZTRkUermn,P2KWvsEWBQ,AgS4FFBg8m,IE0SF6BTEB,lBocTTcnRM,Cqqm8eQ1If,tzT4slvIwR,...,1QvXK4H6Ru,Uwjz8m0YcC,JoLVkDxS44,yRVWYuvtq2,eev7RsNa8P,IkDPzM03Sx,XaNw8hYLjk,ghisamesER,spcaQOLgxo,0vVahjHuuC
4222,xO9WRD2Zbm,qi8qLmgATK,ORXXF0IEnF,X2wn0fgqwH,sAoRKO6PLe,Sm9S0hzVpj,ooHfH960JX,Ibp0sHM5hA,dch1WQaq9W,sIzsew97rH,...,pN9jbu8oeI,ypiaxbYwUe,BFmzhNkgLt,KpiEA8pmw2,r7hfGuks2f,wprAMV2BLw,dOVqOXbGkT,bvXtJiSFJ5,yOhu2oTbSY,WbFJZJVHki
4223,EW403CecBe,kGYz42FXiw,WDebBQgFr8,Arsr663fsf,7da2Fx22If,OOgoguexmz,PTK1h7YWXy,STAdwcSAM1,U0f51ZmLvM,gUQQbWny7I,...,V1tG3CjHuq,VmKW4GjvGO,5mA1Md73xj,lAxFxvgOKQ,pvGUb7hgvt,Q7b1WBHm3R,8JeVX81aZq,k9F8pVYcuc,UqCFYQNiAh,nE2BeBrwMw
